In [5]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import math
import matplotlib.pyplot as plt

%matplotlib inline

In [6]:
#load data from file

data = pd.read_csv('my_data.csv')
data = data.as_matrix()

In [7]:
train = data[:,:11]
train[train==-1] = 0
mean = np.mean(train,axis=0)
mean = mean.reshape(1,11)
for i in range(11):
    train[train[:,i]==0,i] = mean[0,i]
    pass
print(train[:10,:])

[[1 1136070024 203761 138467 1 138371 98652 1 259200 76317.90602113931 1]
 [2 1136070690 9729.77218340525 11 1 4 35848 1 259200 76317.90602113931 1]
 [3 1136071207 117 201203 1 18882.1760718103 254212.10202486045 1 259200
  76317.90602113931 1]
 [4 1136071267 4406 196985 1 18882.1760718103 254212.10202486045 1 259200
  76317.90602113931 1]
 [5 1136071269 202516 19520 1 18731 522268 1 259200 76317.90602113931 1]
 [6 1136072890 1 46 1 3 49216 1 900 76317.90602113931 1]
 [7 1136074263 2629516 21 1 18882.1760718103 35712 1 5400 76317.90602113931
  1]
 [8 1136074695 1 197831 1 18882.1760718103 254212.10202486045 1 259200
  76317.90602113931 1]
 [9 1136074754 2 197682 1 18882.1760718103 254212.10202486045 1 259200
  76317.90602113931 1]
 [10 1136074756 2 197552 1 18882.1760718103 254212.10202486045 1 259200
  76317.90602113931 1]]


In [8]:
print(train[:10,:])

[[1 1136070024 203761 138467 1 138371 98652 1 259200 76317.90602113931 1]
 [2 1136070690 9729.77218340525 11 1 4 35848 1 259200 76317.90602113931 1]
 [3 1136071207 117 201203 1 18882.1760718103 254212.10202486045 1 259200
  76317.90602113931 1]
 [4 1136071267 4406 196985 1 18882.1760718103 254212.10202486045 1 259200
  76317.90602113931 1]
 [5 1136071269 202516 19520 1 18731 522268 1 259200 76317.90602113931 1]
 [6 1136072890 1 46 1 3 49216 1 900 76317.90602113931 1]
 [7 1136074263 2629516 21 1 18882.1760718103 35712 1 5400 76317.90602113931
  1]
 [8 1136074695 1 197831 1 18882.1760718103 254212.10202486045 1 259200
  76317.90602113931 1]
 [9 1136074754 2 197682 1 18882.1760718103 254212.10202486045 1 259200
  76317.90602113931 1]
 [10 1136074756 2 197552 1 18882.1760718103 254212.10202486045 1 259200
  76317.90602113931 1]]


In [9]:
#preprocess data
def preprocess(train):
    data_min = np.min(train,axis=0)
    data_max = np.max(train,axis=0)
    data_min = data_min.reshape(1,11)
    data_max = data_max.reshape(1,11)
    norm = data_max - data_min
    print(data_max)
    print(data_min)
    print(norm)
    train_norm = (train - data_min) / (norm + 1e-5)
    return train_norm

In [10]:
train_data = preprocess(train)

[[404176 1167605790 4345394 1575814 1 259316 3667652 1 345600 1126400 5]]
[[1 1136070024 1 1 0.8600485926922925 1 1700 1 600 76317.90602113931 1]]
[[404175 31535766 4345393 1575813 0.1399514073077075 259315 3665952 0
  345000 1050082.0939788606 4]]


In [11]:
num_training = 10000
num_validation = 1000

X_train = train_data[:11000,:]
mask = range(num_training,num_training + num_validation)
X_val = X_train[mask]
mask = range(num_training)
X_train = X_train[mask]
X_test = train_data[15000:20000,:]

In [12]:

# 4 column is the burst time 
y_train = X_train[:,3].reshape(-1,1)
y_val = X_val[:,3].reshape(-1,1)
y_test = X_test[:,3].reshape(-1,1)

#removing column 4 from input
X_train = np.delete(X_train,3,1)
X_val = np.delete(X_val,3,1)
X_test = np.delete(X_test,3,1)

In [13]:
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  (10000, 10)
Train labels shape:  (10000, 1)
Validation data shape:  (1000, 10)
Validation labels shape:  (1000, 1)
Test data shape:  (5000, 10)
Test labels shape:  (5000, 1)


In [ ]:
BATCH_START = 0
TIME_STEPS = 20
BATCH_SIZE = 50
INPUT_SIZE = 1
OUTPUT_SIZE = 1
CELL_SIZE = 10
LR = 0.006



class LSTMRNN(object):
    def __init__(self, n_steps, input_size, output_size, cell_size, batch_size):
        self.n_steps = n_steps
        self.input_size = input_size
        self.output_size = output_size
        self.cell_size = cell_size
        self.batch_size = batch_size
        with tf.name_scope('inputs'):
            self.xs = tf.placeholder(tf.float32, [None, n_steps, input_size], name='xs')
            self.ys = tf.placeholder(tf.float32, [None, n_steps, output_size], name='ys')
        with tf.variable_scope('in_hidden'):
            self.add_input_layer()
        with tf.variable_scope('LSTM_cell'):
            self.add_cell()
        with tf.variable_scope('out_hidden'):
            self.add_output_layer()
        with tf.name_scope('cost'):
            self.compute_cost()
        with tf.name_scope('train'):
            self.train_op = tf.train.AdamOptimizer(LR).minimize(self.cost)

    def add_input_layer(self,):
        l_in_x = tf.reshape(self.xs, [-1, self.input_size], name='2_2D')  # (batch*n_step, in_size)
        # Ws (in_size, cell_size)
        Ws_in = self._weight_variable([self.input_size, self.cell_size])
        # bs (cell_size, )
        bs_in = self._bias_variable([self.cell_size,])
        # l_in_y = (batch * n_steps, cell_size)
        with tf.name_scope('Wx_plus_b'):
            l_in_y = tf.matmul(l_in_x, Ws_in) + bs_in
        # reshape l_in_y ==> (batch, n_steps, cell_size)
        self.l_in_y = tf.reshape(l_in_y, [-1, self.n_steps, self.cell_size], name='2_3D')

    def add_cell(self):
        lstm_cell = tf.contrib.rnn.BasicLSTMCell(self.cell_size, forget_bias=1.0, state_is_tuple=True)
        with tf.name_scope('initial_state'):
            self.cell_init_state = lstm_cell.zero_state(self.batch_size, dtype=tf.float32)
        self.cell_outputs, self.cell_final_state = tf.nn.dynamic_rnn(
            lstm_cell, self.l_in_y, initial_state=self.cell_init_state, time_major=False)

    def add_output_layer(self):
        # shape = (batch * steps, cell_size)
        l_out_x = tf.reshape(self.cell_outputs, [-1, self.cell_size], name='2_2D')
        Ws_out = self._weight_variable([self.cell_size, self.output_size])
        bs_out = self._bias_variable([self.output_size, ])
        # shape = (batch * steps, output_size)
        with tf.name_scope('Wx_plus_b'):
            self.pred = tf.matmul(l_out_x, Ws_out) + bs_out

    def compute_cost(self):
        losses = tf.contrib.legacy_seq2seq.sequence_loss_by_example(
            [tf.reshape(self.pred, [-1], name='reshape_pred')],
            [tf.reshape(self.ys, [-1], name='reshape_target')],
            [tf.ones([self.batch_size * self.n_steps], dtype=tf.float32)],
            average_across_timesteps=True,
            softmax_loss_function=self.ms_error,
            name='losses'
        )
        with tf.name_scope('average_cost'):
            self.cost = tf.div(
                tf.reduce_sum(losses, name='losses_sum'),
                self.batch_size,
                name='average_cost')
            tf.summary.scalar('cost', self.cost)

    @staticmethod
    def ms_error(labels, logits):
        return tf.square(tf.subtract(labels, logits))

    def _weight_variable(self, shape, name='weights'):
        initializer = tf.random_normal_initializer(mean=0., stddev=1.,)
        return tf.get_variable(shape=shape, initializer=initializer, name=name)

    def _bias_variable(self, shape, name='biases'):
        initializer = tf.constant_initializer(0.1)
        return tf.get_variable(name=name, shape=shape, initializer=initializer)


if __name__ == '__main__':
    model = LSTMRNN(TIME_STEPS, INPUT_SIZE, OUTPUT_SIZE, CELL_SIZE, BATCH_SIZE)
    sess = tf.Session()
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter("logs", sess.graph)
    # tf.initialize_all_variables() no long valid from
    # 2017-03-02 if using tensorflow >= 0.12
    if int((tf.__version__).split('.')[1]) < 12 and int((tf.__version__).split('.')[0]) < 1:
        init = tf.initialize_all_variables()
    else:
        init = tf.global_variables_initializer()
    sess.run(init)
    # relocate to the local dir and run this line to view it on Chrome (http://0.0.0.0:6006/):
    # $ tensorboard --logdir='logs'

    plt.ion()
    plt.show()
    
    
    Xd = X_train
    yd = y_train
    
    for i in range(int(math.ceil(Xd.shape[0]/BATCH_SIZE))):
        # generate indicies for the batch
        start_idx = (i*BATCH_SIZE)%Xd.shape[0]
        idx = train_indicies[start_idx:start_idx+BATCH_SIZE]
        
        
        if i == 0:
            
            
            feed_dict = {
                    model.xs: Xd[idx,:],
                    model.ys: yd[idx],
                    # create initial state
            }
        else:
            feed_dict = {
                model.xs: Xd[idx,:],
                model.ys: yd[idx],
                model.cell_init_state: state    # use last state as the initial state for this run
            }

        _, cost, state, pred = sess.run(
            [model.train_op, model.cost, model.cell_final_state, model.pred],
            feed_dict=feed_dict)

        # plotting
        #plt.plot(xs[0, :], res[0].flatten(), 'r', xs[0, :], pred.flatten()[:TIME_STEPS], 'b--')
        #plt.ylim((-1.2, 1.2))
        #plt.draw()
        #plt.pause(0.3)

#        if i % 20 == 0:
 #           print('cost: ', round(cost, 4))
  #          result = sess.run(merged, feed_dict)
#writer.add_summary(result, i)



